## Question 1, Answer: D
### Problem:
#### Recall that N is the size of the data set and d is the dimensionality of the
#### input space. The original formulation of the hard-margin SVM problem (minimize 0.5 *
#### wTw subject to the inequality constraints), without going through the
#### Lagrangian dual problem, is
### Solution:
#### If we are trying to minimize 0.5wTw, then we are effectively trying to minimize the magnitude of w
#### w corresponds to the d input features plus the bias. This leads to w having d + 1 parameters
#### Hence, the quadratic programming problem has d + 1 variables.

## Question 2, Answer:  A
#### The kernel for problem 2 is of the form $K(x_n, x_m) = (1 + x_n^Tx_m)^Q$
#### We can use the SVM classifier from the scikit-learn package and specify the required regularization constant C and kernel
#### The SVM classifier uses a polynomial kernel of this form $K(x, x') = (\gamma\langle x, x' \rangle + r)^d$, where $\langle x, x' \rangle$ specifies the inner product between $x$ and $x'$
#### We can also choose $\gamma = r= 1,d = Q$ to match our required kernel

In [69]:
# Importing necessary libraries
import numpy as np
from sklearn import svm, model_selection
from statistics import mode

In [6]:
# loading datasets using numpy
def load_datasets():
    train_ds = np.loadtxt('features.train')
    test_ds = np.loadtxt('features.test')
    return train_ds, test_ds


# getting a data set with labels 1 for n and -1 otherwise
def get_masked_ds(ds, n, m=None):
    # mark all rows with n with 1
    n_mask = (ds[:, 0] == n)
    ds[n_mask, 0] = 1
    
    # -1 labels
    if m:
        # only label rows with m as -1
        m_mask = (ds[:, 0] == m)
        ds[m_mask, 0] = -1
        
         # discarding rows that aren't either m or n
        ds = ds[(n_mask | m_mask)]
    else:
        # mark all other rows as -1
        ds[~n_mask, 0] = -1
    
    # returning X(features), y(labels)
    X = ds[:, 1:]
    y = ds[:, 0]
    return X, y


# get X and y I/O pairs for n vs m classification
# if m is not given, then n vs all is default assumption for classification
def get_n_vs_m_ds(n, m=None):
    # get train and test ds
    train_ds, test_ds = load_datasets()
    
    # given the number n, mark all rows that are of the number n with 1
    # and mark all other rows with -1
    X_train, y_train = get_masked_ds(train_ds, n, m)
    X_test, y_test = get_masked_ds(test_ds, n, m)
    
    # returns
    return (X_train, y_train), (X_test, y_test)


# run n vs all classification SVM with specified kernels for given n values
# return a dictionary with n as key and (E_in, num_support_vecs) as a value
def get_n_vs_all_classification_info(n_values, C_reg, Q):
    n_dict = {}
    for n in n_values: # iterates over different n
        # gets dataset
        (X, y), _ = get_n_vs_m_ds(n)
        
        # fits the data
        classifier = svm.SVC(C=C_reg, kernel='poly', gamma=1, degree=Q, coef0=1) # getting configured SVM with polynomial kernel
        classifier.fit(X, y)
        
        # evaluating in sample error
        y_in_predict = classifier.predict(X)
        E_in = np.mean(y_in_predict != y)
        
        # to return result
        n_dict[n] = (np.round(E_in, 2), classifier.support_vectors_.shape[0])   
    return n_dict


# evaluates classification of polynomial kernel for n vs m classification for varying degrees of kernel, and varying regularization constants
def get_n_vs_m_classification_info(n, m, C_reg_vals, Q_vals):
    n_m_dict = {}
    
    # getting datasets
    (X_tr, y_tr), (X_te, y_te) = get_n_vs_m_ds(n, m)
    
    for C_reg in C_reg_vals: # iterate over C
        for Q in Q_vals: # iterate over Q
            # fitting the data
            classifier = svm.SVC(C=C_reg, kernel='poly', gamma=1, degree=Q, coef0=1) # getting configured SVM with polynomial kernel
            classifier.fit(X_tr, y_tr)
            
            # in sample error calc
            y_in_predict = classifier.predict(X_tr)
            E_in = np.mean(y_in_predict != y_tr)
            
            # out of sample error calc
            y_out_predict = classifier.predict(X_te)
            E_out = np.mean(y_out_predict != y_te)
            
            # to return result
            n_m_dict[(n, m, C_reg, Q)] = (np.round(E_in, 4), np.round(E_out, 4), classifier.support_vectors_.shape[0])
    return n_m_dict

In [8]:
# Question 2
# specifying n vs all in values for n
# regularization constant C_reg
# polynomial kernel degree Q
classification_info = get_n_vs_all_classification_info(n_values=(0, 2, 4, 6, 8), C_reg=0.01, Q=2) # contains in sample error/number of support vecs for each n vs all
for n, info in classification_info.items():
    print(f'{n} vs all had an in sample error of {info[0]} with {info[1]} support vectors')
print('\nAs we can clearly see, 0 vs all had the highest in sample error of 0.11 with 2179 support vectors')

0 vs all had an in sample error of 0.11 with 2179 support vectors
2 vs all had an in sample error of 0.1 with 1970 support vectors
4 vs all had an in sample error of 0.09 with 1856 support vectors
6 vs all had an in sample error of 0.09 with 1893 support vectors
8 vs all had an in sample error of 0.07 with 1776 support vectors

As we can clearly see, 0 vs all had the highest in sample error of 0.11 with 2179 support vectors


## Question 3, Answer: A
#### We are going to employ the same tactic as we did in problem 2

In [4]:
# Question 3
# specifying n vs all in values for n
# regularization constant C_reg
# polynomial kernel degree Q
classification_info = get_n_vs_all_classification_info(n_values=(1, 3, 5, 7, 9), C_reg=0.01, Q=2) # contains in sample error/number of support vecs for each n vs all
for n, info in classification_info.items():
    print(f'{n} vs all had an in sample error of {info[0]} with {info[1]} support vectors')
print('\nAs we can clearly see, 1 vs all had the lowest in sample error of 0.01 with 386 support vectors')

1 vs all had an in sample error of 0.01 with 386 support vectors
3 vs all had an in sample error of 0.09 with 1950 support vectors
5 vs all had an in sample error of 0.08 with 1585 support vectors
7 vs all had an in sample error of 0.09 with 1704 support vectors
9 vs all had an in sample error of 0.09 with 1978 support vectors

As we can clearly see, 1 vs all had the lowest in sample error of 0.01 with 386 support vectors


## Question 4, Answer: C

In [5]:
difference = 2179 - 386
print(f'There was a difference of {difference}  support vectors from the selected classifiers from 1 and 2')
print('This is closest to answer choice C, 1800 support vectors')

There was a difference of 1793  support vectors from the selected classifiers from 1 and 2
This is closest to answer choice C, 1800 support vectors


## Question 5, Answer: D

In [17]:
classification_info = get_n_vs_m_classification_info(n=1, m=5, C_reg_vals=[0.001, 0.01, 0.1, 1], Q_vals=[2])
for key, value in classification_info.items():
    _, _, C_reg, _ = key
    E_in, E_out, num_support_vectors = value
    print(f'Using regularization constant C={C_reg} had in sample error {E_in} and out of sample error {E_out} with {num_support_vectors} support vectors\n')
    
print('\n\nWe can clearly see a maximum C=1 achieves the lowest E_in = 0.0032. This means that Answer D is correct')

Using regularization constant C=0.001 had in sample error 0.0045 and out of sample error 0.0165 with 76 support vectors

Using regularization constant C=0.01 had in sample error 0.0045 and out of sample error 0.0189 with 34 support vectors

Using regularization constant C=0.1 had in sample error 0.0045 and out of sample error 0.0189 with 24 support vectors

Using regularization constant C=1 had in sample error 0.0032 and out of sample error 0.0189 with 24 support vectors



We can clearly see a maximum C=1 achieves the lowest E_in = 0.0032. This means that Answer D is correct


## Question 6, Answer: B 

In [23]:
classification_info = get_n_vs_m_classification_info(n=1, m=5, C_reg_vals=[0.0001, 0.001, 0.01, 1], Q_vals=[2, 5])
for key, value in classification_info.items():
    _, _, C_reg, Q = key
    E_in, E_out, num_support_vectors = value
    print(f'Using regularization constant C={C_reg} and kernel degree Q ={Q} had in sample error {E_in} and out of sample error {E_out} with {num_support_vectors} support vectors\n\n')
    
print('\n\nWe can clearly see that when C=0.001, the number of support vectors is lower when Q = 5. This means that Answer B is correct')

Using regularization constant C=0.0001 and kernel degree Q =2 had in sample error 0.009 and out of sample error 0.0165 with 236 support vectors


Using regularization constant C=0.0001 and kernel degree Q =5 had in sample error 0.0045 and out of sample error 0.0189 with 26 support vectors


Using regularization constant C=0.001 and kernel degree Q =2 had in sample error 0.0045 and out of sample error 0.0165 with 76 support vectors


Using regularization constant C=0.001 and kernel degree Q =5 had in sample error 0.0045 and out of sample error 0.0212 with 25 support vectors


Using regularization constant C=0.01 and kernel degree Q =2 had in sample error 0.0045 and out of sample error 0.0189 with 34 support vectors


Using regularization constant C=0.01 and kernel degree Q =5 had in sample error 0.0038 and out of sample error 0.0212 with 23 support vectors


Using regularization constant C=1 and kernel degree Q =2 had in sample error 0.0032 and out of sample error 0.0189 with 24 support

## Question 7, Answer: B
## Question 8, Answer: C

In [62]:
# Calculate E_cv by randomly partitioning training dataset into 10 portions
# do 10 seperate training sessions, choosing 1 of the portions as the validation set
# calculate cross validation error as average of 10 validation errors
def cross_validation_model_selection(n, m, C_vals=[0.0001, 0.001, 0.01, 0.1, 1], Q=2):
    # 10 fold data splitting indices
    k_fold = model_selection.StratifiedKFold(n_splits=10, shuffle=True)
    
    # getting dataset, only training
    (X_tr, y_tr), _ = get_n_vs_m_ds(n, m)
    
    E_vals_per_C = {}
    
    # training on 10 uniquely different partitions of data
    for train_index, test_index in k_fold.split(X_tr, y_tr):
        for C_reg in C_vals:
            X_train, X_val = X_tr[train_index], X_tr[test_index]
            y_train, y_val = y_tr[train_index], y_tr[test_index]

            classifier = svm.SVC(C=C_reg, kernel='poly', gamma=1, degree=Q, coef0=1) # getting configured SVM with polynomial kernel
            classifier.fit(X_train, y_train)

            y_val_predict = classifier.predict(X_val)
            E_val = np.mean(y_val_predict != y_val)

            if C_reg not in E_vals_per_C:
                E_vals_per_C[C_reg] = [E_val]
            else:
                E_vals_per_C[C_reg].append(E_val)
        
    # calculating cross validation error for each C
    E_cv_per_C = {}
    for C_reg, E_vals in E_vals_per_C.items():
        E_cv_per_C[C_reg] = np.mean(E_vals)
    
    # finding the C with the minimum cross validation error and returning it
    min_E_cv = min(E_cv_per_C.values())
    for C in C_vals:
        if E_cv_per_C[C] == min_E_cv:
            return (C, min_E_cv)
    


In [74]:
selected_Cs = []
E_cv_vals = {}

# performing experiment 100 times
for i in range(100):
    C_selected, E_cv = cross_validation_model_selection(1, 5) # for 1 vs 5
    selected_Cs.append(C_selected)
    if C_selected not in E_cv_vals:
        E_cv_vals[C_selected] = [E_cv]
    else:
        E_cv_vals[C_selected].append(E_cv)

# finding most selected C and computing the average cross validation error for it
most_selected_C = mode(selected_Cs)
E_cv_avg = np.round(np.mean(E_cv_vals[most_selected_C]), 4)
print(f'The most commonly selected regularization parameter was C={most_selected_C} with an average cross validation error of E_cv={E_cv_avg}')
print(f'This points us to answer choice B for question 7 and answer choice C for question 8')

The most commonly selected regularization parameter was C=0.001 with an average cross validation error of E_cv=0.0046
This points us to answer choice B for question 7 and answer choice C for question 8


## Question 9, Answer: E
## Question 10, Answer: C

In [82]:
# returns classification performance for n vs m for RBF kernel choice for varying regularization constants
def get_n_vs_m_classification_info_rbf(n, m, C_reg_vals=(0.01, 1, 100, 10**4, 10**6)):
    n_m_dict = {}
    (X_tr, y_tr), (X_te, y_te) = get_n_vs_m_ds(n, m) # dataset
    
    for C_reg in C_reg_vals: # iterate over C vals
            # fitting data
            classifier = svm.SVC(C=C_reg, kernel='rbf', gamma=1) # getting configured SVM with rbf kernel
            classifier.fit(X_tr, y_tr)
            
            # in sample error
            y_in_predict = classifier.predict(X_tr)
            E_in = np.mean(y_in_predict != y_tr)

            # out of sample error
            y_out_predict = classifier.predict(X_te)
            E_out = np.mean(y_out_predict != y_te)
            
            # to return result
            n_m_dict[C_reg] = (np.round(E_in, 4), np.round(E_out, 4), classifier.support_vectors_.shape[0])
    return n_m_dict

In [85]:
rbf_classifciation_info = get_n_vs_m_classification_info_rbf(1, 5)
for C_reg, value in rbf_classifciation_info.items():
    print(f'For C_reg={C_reg}, E_in={value[0]}, E_out = {value[1]}\n')
print('As we can see the minimum in sample error E_in is achieved for C = 10^6.\n This points us to answer choice E for question 9')
print('As we can see the minimum out of sample error E_out is achieved for C = 100. This points us to answer choice C for question 10')

For C_reg=0.01, E_in=0.0038, E_out = 0.0236

For C_reg=1, E_in=0.0045, E_out = 0.0212

For C_reg=100, E_in=0.0032, E_out = 0.0189

For C_reg=10000, E_in=0.0026, E_out = 0.0236

For C_reg=1000000, E_in=0.0006, E_out = 0.0236

As we can see the minimum in sample error E_in is achieved for C = 10^6.
 This points us to answer choice E for question 9
As we can see the minimum out of sample error E_out is achieved for C = 100. This points us to answer choice C for question 10
